<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/%EC%A6%9D%EB%B6%84C%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC_epoch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=273e06db2fc9ebb7f2003af5caf355d1505c9379ffa0a1b462d93742219d8145
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 769 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 86.1 MB/s 
     |████████████████████████████████| 895 kB 72.0 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l5ba8lcu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [4]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/df_C_inc.csv')

In [5]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [6]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000044,C,25,259,금속재료 주문에 의한 절삭가공 공작기계
1,id_0000132,C,25,259,구입한 쇠를 깎고. 연마해서 기계부품 생산
2,id_0000239,C,25,259,철판 프레스가공 모터에 들어가는 코아
3,id_0000252,C,25,259,제조 기계가공 전자부품
4,id_0000372,C,25,259,가공된 금속재료로 절삭가공하여 선반 제조
5,id_0000458,C,25,259,와이어를 절단 가공하여 자동차 부품 제조
6,id_0000520,C,25,259,"철판, 프레스 가공 방열문 부품 생산"
7,id_0000695,C,25,259,"철 절삭, 용접하여 초경 절삭공구"
8,id_0000864,C,25,259,"제조 기계부품, 금속단조제품"
9,id_0000924,C,25,259,그 외 기타 분류 안된 금속가공 제품 제조업


In [8]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [9]:
df.shape

(830871, 5)

In [10]:
df['digit_3'].value_counts()

152    15420
131    14220
259    12876
105    12800
241    12790
       ...  
304     4400
301     4000
191     4000
266     1600
120      600
Name: digit_3, Length: 85, dtype: int64

In [11]:
df.loc[(df['digit_3']==101), 'digit_3']=0
df.loc[(df['digit_3']==102), 'digit_3']=1
df.loc[(df['digit_3']==103), 'digit_3']=2
df.loc[(df['digit_3']==104), 'digit_3']=3
df.loc[(df['digit_3']==105), 'digit_3']=4
df.loc[(df['digit_3']==106), 'digit_3']=5
df.loc[(df['digit_3']==107), 'digit_3']=6
df.loc[(df['digit_3']==108), 'digit_3']=7
df.loc[(df['digit_3']==111), 'digit_3']=8
df.loc[(df['digit_3']==112), 'digit_3']=9
df.loc[(df['digit_3']==120), 'digit_3']=10
df.loc[(df['digit_3']==131), 'digit_3']=11
df.loc[(df['digit_3']==132), 'digit_3']=12
df.loc[(df['digit_3']==133), 'digit_3']=13
df.loc[(df['digit_3']==134), 'digit_3']=14
df.loc[(df['digit_3']==139), 'digit_3']=15
df.loc[(df['digit_3']==141), 'digit_3']=16
df.loc[(df['digit_3']==142), 'digit_3']=17
df.loc[(df['digit_3']==143), 'digit_3']=18
df.loc[(df['digit_3']==144), 'digit_3']=19
df.loc[(df['digit_3']==151), 'digit_3']=20
df.loc[(df['digit_3']==152), 'digit_3']=21
df.loc[(df['digit_3']==161), 'digit_3']=22
df.loc[(df['digit_3']==162), 'digit_3']=23
df.loc[(df['digit_3']==163), 'digit_3']=24
df.loc[(df['digit_3']==171), 'digit_3']=25
df.loc[(df['digit_3']==172), 'digit_3']=26
df.loc[(df['digit_3']==179), 'digit_3']=27
df.loc[(df['digit_3']==181), 'digit_3']=28
df.loc[(df['digit_3']==182), 'digit_3']=29
df.loc[(df['digit_3']==191), 'digit_3']=30
df.loc[(df['digit_3']==192), 'digit_3']=31
df.loc[(df['digit_3']==201), 'digit_3']=32
df.loc[(df['digit_3']==202), 'digit_3']=33
df.loc[(df['digit_3']==203), 'digit_3']=34
df.loc[(df['digit_3']==204), 'digit_3']=35
df.loc[(df['digit_3']==205), 'digit_3']=36
df.loc[(df['digit_3']==211), 'digit_3']=37
df.loc[(df['digit_3']==212), 'digit_3']=38
df.loc[(df['digit_3']==213), 'digit_3']=39
df.loc[(df['digit_3']==221), 'digit_3']=40
df.loc[(df['digit_3']==222), 'digit_3']=41
df.loc[(df['digit_3']==231), 'digit_3']=42
df.loc[(df['digit_3']==232), 'digit_3']=43
df.loc[(df['digit_3']==233), 'digit_3']=44
df.loc[(df['digit_3']==239), 'digit_3']=45
df.loc[(df['digit_3']==241), 'digit_3']=46
df.loc[(df['digit_3']==242), 'digit_3']=47
df.loc[(df['digit_3']==243), 'digit_3']=48
df.loc[(df['digit_3']==251), 'digit_3']=49
df.loc[(df['digit_3']==252), 'digit_3']=50
df.loc[(df['digit_3']==259), 'digit_3']=51
df.loc[(df['digit_3']==261), 'digit_3']=52
df.loc[(df['digit_3']==262), 'digit_3']=53
df.loc[(df['digit_3']==263), 'digit_3']=54
df.loc[(df['digit_3']==264), 'digit_3']=55
df.loc[(df['digit_3']==265), 'digit_3']=56
df.loc[(df['digit_3']==266), 'digit_3']=57
df.loc[(df['digit_3']==271), 'digit_3']=58
df.loc[(df['digit_3']==272), 'digit_3']=59
df.loc[(df['digit_3']==273), 'digit_3']=60
df.loc[(df['digit_3']==274), 'digit_3']=61
df.loc[(df['digit_3']==281), 'digit_3']=62
df.loc[(df['digit_3']==282), 'digit_3']=63
df.loc[(df['digit_3']==283), 'digit_3']=64
df.loc[(df['digit_3']==284), 'digit_3']=65
df.loc[(df['digit_3']==285), 'digit_3']=66
df.loc[(df['digit_3']==289), 'digit_3']=67
df.loc[(df['digit_3']==291), 'digit_3']=68
df.loc[(df['digit_3']==292), 'digit_3']=69
df.loc[(df['digit_3']==301), 'digit_3']=70
df.loc[(df['digit_3']==302), 'digit_3']=71
df.loc[(df['digit_3']==303), 'digit_3']=72
df.loc[(df['digit_3']==304), 'digit_3']=73
df.loc[(df['digit_3']==311), 'digit_3']=74
df.loc[(df['digit_3']==312), 'digit_3']=75
df.loc[(df['digit_3']==313), 'digit_3']=76
df.loc[(df['digit_3']==319), 'digit_3']=77
df.loc[(df['digit_3']==320), 'digit_3']=78
df.loc[(df['digit_3']==331), 'digit_3']=79
df.loc[(df['digit_3']==332), 'digit_3']=80
df.loc[(df['digit_3']==333), 'digit_3']=81
df.loc[(df['digit_3']==334), 'digit_3']=82
df.loc[(df['digit_3']==339), 'digit_3']=83
df.loc[(df['digit_3']==340), 'digit_3']=84

In [12]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['금속재료 주문에 의한 절삭가공 공작기계', '51'], ['구입한 쇠를 깎고. 연마해서 기계부품 생산', '51'], ['철판 프레스가공 모터에 들어가는 코아', '51'], ['제조 기계가공 전자부품', '51'], ['가공된 금속재료로 절삭가공하여 선반 제조', '51']]


## Train data & test data

In [15]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [18]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [20]:
data_train[0]

(array([   2, 1479, 6305, 2929, 7767, 7512, 4151,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(8, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 56)

In [21]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [22]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=85,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [24]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/11685 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 4.535937309265137 train acc 0.0
epoch 1 batch id 201 loss 4.462684631347656 train acc 0.01127176616915423
epoch 1 batch id 401 loss 4.435065269470215 train acc 0.01258572319201995
epoch 1 batch id 601 loss 4.31093168258667 train acc 0.016950915141430947
epoch 1 batch id 801 loss 4.232960224151611 train acc 0.02787531210986267
epoch 1 batch id 1001 loss 3.8765437602996826 train acc 0.046235014985014984
epoch 1 batch id 1201 loss 3.432330369949341 train acc 0.07081338467943381
epoch 1 batch id 1401 loss 3.2096877098083496 train acc 0.10039703783012134
epoch 1 batch id 1601 loss 3.0491483211517334 train acc 0.13412515615240475
epoch 1 batch id 1801 loss 2.3773324489593506 train acc 0.16947182121043863
epoch 1 batch id 2001 loss 2.3052685260772705 train acc 0.20313280859570215
epoch 1 batch id 2201 loss 2.192168712615967 train acc 0.23487193321217628
epoch 1 batch id 2401 loss 1.9410693645477295 train acc 0.26433647438567265
epoch 1 batch id 2601 loss 1.668269038200

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9332298883756736


  0%|          | 0/11685 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.039256613701581955 train acc 1.0
epoch 5 batch id 201 loss 0.05992460995912552 train acc 0.9479944029850746
epoch 5 batch id 401 loss 0.11237835884094238 train acc 0.9480595386533666
epoch 5 batch id 601 loss 0.33058398962020874 train acc 0.9475873544093179
epoch 5 batch id 801 loss 0.1907873898744583 train acc 0.9473899812734082
epoch 5 batch id 1001 loss 0.18716977536678314 train acc 0.9474744005994006
epoch 5 batch id 1201 loss 0.2964102029800415 train acc 0.9472705037468776
epoch 5 batch id 1401 loss 0.13530710339546204 train acc 0.9469463775874375
epoch 5 batch id 1601 loss 0.4966737926006317 train acc 0.9465178013741412
epoch 5 batch id 1801 loss 0.13349860906600952 train acc 0.94676568573015
epoch 5 batch id 2001 loss 0.19214800000190735 train acc 0.9469249750124937
epoch 5 batch id 2201 loss 0.08396352827548981 train acc 0.9468849386642435
epoch 5 batch id 2401 loss 0.209308922290802 train acc 0.9472029883381924
epoch 5 batch id 2601 loss 0.18880979716

  0%|          | 0/1299 [00:00<?, ?it/s]

epoch 5 test acc 0.9364294649730562


In [25]:
#학습 모델 저장
torch.save(model, 'C_inc_model.pt')